<a href="https://colab.research.google.com/github/sadat1971/Llama2_custom_finetuning/blob/main/Barbie_QA_chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install langchain_openai

In [ ]:
!pip install langchain

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import langchain
from langchain_openai import OpenAI
import re
from typing import List, Tuple


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = '******************************'

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct")

We will have three parts of QA:

1. First part focused on **Intro**: around 300 QA
2. Second part focused on **Plot**: around 600 QA

*   100 on first 2 paragraph
*   300 on middle 3 paargaphs
*   100 on last 3 paragraphs

3. Third part focused on the **Cast**: around 100 QA



In [ ]:
intro_description = f"""Barbie is a 2023 fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the eponymous fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film stars Margot Robbie as the title character and Ryan Gosling as Ken, and follows the pair on a journey of self-discovery through both Barbieland and the real world following an existential crisis. The supporting cast includes America Ferrera, Michael Cera, Kate McKinnon, Issa Rae, Rhea Perlman, and Will Ferrell.

A live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 2018. Robbie was cast in 2019, after Gal Gadot turned down the role due to scheduling conflicts, and Gerwig was announced as director and co-writer with Baumbach in 2020. The rest of the cast was announced in early 2022, with principal photography occurring primarily at Warner Bros. Studios, Leavesden, in England and at the Venice Beach Skatepark in Los Angeles from March to July 2022.

Barbie premiered at the Shrine Auditorium in Los Angeles on July 9, 2023, and was released in the United States on July 21. Its simultaneous release with Universal's Oppenheimer led to the Barbenheimer cultural phenomenon, which encouraged audiences to see both films as a double feature. The film has grossed $1.44 billion and achieved several milestones, including becoming the highest-grossing film of 2023 and the 14th highest-grossing film of all time. Named one of the top 10 films of 2023 by the National Board of Review and the American Film Institute, it received critical acclaim and other accolades, including eight Academy Award nominations (including Best Picture) and won two Golden Globe Awards.
"""

plot_description = f"""Stereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland, a matriarchal society populated by different versions of Barbies, Kens, and a group of discontinued models who are treated like outcasts due to their unconventional traits. While the Kens spend their days playing at the beach, considering it their profession, the Barbies hold prestigious jobs in science, politics, media, and so on. Beach Ken ("Ken") is only happy when he is with Barbie, and seeks a closer relationship, but she rebuffs him in favor of other activities and female friendships.

One evening at a dance party, Barbie is suddenly stricken with worries about mortality. Overnight, she develops bad breath, cellulite, and flat feet, disrupting her usual routines and the classic perfection of the Barbies the next day. She seeks out Weird Barbie, a disfigured doll, who tells her she must find the child playing with her in the real world to cure her afflictions. Ken stows away in her convertible to join her, to which Barbie reluctantly agrees.

Arriving at Venice Beach, Barbie punches a man for groping her, leading to her and Ken's brief arrest. Alarmed by their presence, Mattel's CEO orders their recapture. Barbie tracks down her owner, a teenage girl named Sasha, who criticizes her for encouraging unrealistic beauty standards. Distraught, Barbie discovers that Gloria, a Mattel employee and Sasha's mother, inadvertently caused her existential crisis after Gloria began playing with Sasha's old Barbie toys. Mattel attempts to put Barbie in a toy box for remanufacturing, but she escapes with Gloria and Sasha's help, and the three travel to Barbieland with Mattel executives in pursuit.

Meanwhile, Ken learns about patriarchy and feels respected for the first time. Returning to Barbieland before Barbie does, he persuades the other Kens to take over, and the Barbies are indoctrinated into submissive roles, such as agreeable girlfriends, housewives, and maids. Barbie arrives and fails to convince everyone to return to the way things were. She becomes depressed, but Gloria gives her a speech about society's conflicting expectations of women, restoring Barbie's self-confidence.

With the assistance of Sasha, Weird Barbie, Allan, and the discontinued dolls, Gloria's speech deprograms the Barbies from their indoctrination. They then manipulate the Kens into fighting among themselves, distracting them from enshrining male superiority into Barbieland's constitution, and the Barbies regain power. Having now experienced systemic oppression for themselves, the Barbies resolve to rectify the faults of their previous society, emphasizing better treatment of the Kens and all outcasts.

Barbie and Ken apologize to each other, acknowledging their mistakes. Ken bemoans his lack of purpose without Barbie, so she encourages him to find an autonomous identity. Barbie, who remains unsure of her own identity, meets with the spirit of Ruth Handler, Mattel co-founder and creator of the Barbie doll, who explains that Barbie's story has no set ending and her ever-evolving history surpasses her roots.

Barbie decides to become human and return to the real world and is bidden goodbye by the Barbies, Kens, and Mattel executives. Sometime later, Gloria, her husband, and Sasha take Barbie, now going by the name "Barbara Handler", to her first gynecologist appointment.
"""

cast_description = f"""Margot Robbie at the 2016 San Diego Comic Con International in San Diego, California.
Ryan Gosling at the 2017 San Diego Comic Con International in San Diego, California.
Margot Robbie and Ryan Gosling portray Barbie and Ken.
Margot Robbie as Barbie, often referred to as "Stereotypical Barbie"[10][11]
Main variations of Barbie played by:
Issa Rae as President Barbie[12][13]
Kate McKinnon as Weird Barbie[14][13]
Alexandra Shipp as Writer Barbie[15][13]
Emma Mackey as Physicist Barbie[16][13]
Hari Nef as Dr. Barbie[13]
Sharon Rooney as Lawyer Barbie[12]
Ana Cruz Kayne as Judge Barbie[12][13]
Ritu Arya as Journalist Barbie[12][13]
Dua Lipa as Mermaid Barbie[17][13]
Nicola Coughlan as Diplomat Barbie[12][13]
Ryan Gosling as Ken, often referred to as "Beach Ken"[18]
Main variations of Ken played by:
Simu Liu as Tourist Ken/"Rival Ken"[19][20][13]
Kingsley Ben-Adir as Basketball Ken[12]
Ncuti Gatwa as Artist Ken[19][12]
Scott Evans as Stereotypical Ken[12]
John Cena as Kenmaid, a merman Ken[21][22]
America Ferrera as Gloria, a Mattel employee who helps Barbie in the real world[23]
Ariana Greenblatt as Sasha, Gloria's daughter[24][25]
Rhea Perlman as Ruth Handler, a co-founder of Mattel[5]
Helen Mirren as the narrator[26]
Will Ferrell as the CEO of Mattel[27]
Michael Cera as Allan[28]
Connor Swindells as Aaron Dinkins, a low-level Mattel employee
Jamie Demetriou as the CFO of Mattel
Emerald Fennell as Midge[29]
Asim Chaudhry as a Mattel warehouse employee
Ray Fearon as Dan at the FBI
Erica Ford as Skipper[30]
Hannah Khalique-Brown as "Growing Up" Skipper[31]
Mette Narrative as Barbie Video Girl
Marisa Abela as Teen Talk Barbie[32]
Lucy Boynton as Proust Barbie
Rob Brydon as Sugar Daddy Ken[12]
Tom Stourton as Earring Magic Ken[33]
Ann Roth as the woman on the bench[34]
Annie Mumolo as Anxiety Mom[35]
Lauren Holt as Time Mom[36][35]
Ryan Piers Williams as Gloria's husband"""

Simply copied from https://en.wikipedia.org/wiki/Barbie_(film)

Now, Let's design our prompt

In [ ]:
focus = None #can be introductory, plot (start/middle/end) or cast
describe = None #can be intro_description, plot description or cast_description

In [ ]:
prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs about the movie
Barbie. Make sure the Instruction-Response are in the json format:\n\n
### Example: {{"Instruction": "Tell me what is Barbie movie about", "Response": "It is about..."}}\n\n
### Movie Description:{describe}\n\n
### Response:"""

In [ ]:
print(prompt)

### Instruction: Based on the None information of the movie "Barbie" below, generate 5 instruction-detailed response pairs about the movie
Barbie. Make sure the Instruction-Response are in the json format:


### Example: {"Instruction": "Tell me what is Barbie movie about", "Response": "It is about..."}


### Movie Description:None


### Response:


We also want a way to extract the responses from the string

In [ ]:
def extract_instruction_response_pairs(string: str)-> Tuple[List[str], List[str]]:
    """
    Extracts pairs of instructions and responses from a JSON-formatted string.

    Parameters:
        - json_string (str): A string containing JSON-formatted instruction and response pairs.

    Returns:
        - instructions (list): A list of extracted instructions.
        - responses (list): A list of extracted responses corresponding to the instructions.
    """

    pattern = r'{"Instruction": "(.*?)", "Response": "(.*?)"}'

    # Use re.findall to extract matches
    matches = re.findall(pattern, string)

    # Extract lists of "Instruction" and "Response"
    instructions = [match[0] for match in matches]
    responses = [match[1] for match in matches]

    return instructions, responses

In [ ]:
All_instructions = []
All_reponses = []

In [ ]:
## Generating based on the intro section
start = time.time()
for idx in tqdm(range(60)):
  focus = "introductory"
  describe = intro_description
  prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs.
  Make sure the Instruction-Response are in the json format:\n\n
  ### Example: {{"Instruction": "the instruction", "Response": "the response"}}\n\n
  ### Description:{describe}\n\n
  ### Response:"""
  generated_text = llm(prompt)
  ins, res = extract_instruction_response_pairs(generated_text)
  All_instructions.extend(ins)
  All_reponses.extend(res)

print("\n\n===Time: {} seconds===".format(time.time()-start))

  0%|          | 0/60 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 60/60 [02:20<00:00,  2.34s/it]



===Time: 140.6080949306488 seconds===


In [ ]:
len(All_instructions)

298

Yes, it's not always perfect :)

In [ ]:
## Generating based on the beginning plot section
for idx in tqdm(range(20)):
  focus = "first 2 paragraph of plot"
  describe = plot_description
  prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs.
  Make sure the Instruction-Response are in the json format:\n\n
  ### Example: {{"Instruction": "the instruction", "Response": "the response"}}\n\n
  ### Description:{describe}\n\n
  ### Response:"""
  generated_text = llm(prompt)
  ins, res = extract_instruction_response_pairs(generated_text)
  All_instructions.extend(ins)
  All_reponses.extend(res)

100%|██████████| 20/20 [00:47<00:00,  2.40s/it]


In [ ]:
## Generating based on the middle plot section
for idx in tqdm(range(60)):
  focus = "middle 3 paragraph of plot"
  describe = plot_description
  prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs.
  Make sure the Instruction-Response are in the json format:\n\n
  ### Example: {{"Instruction": "the instruction", "Response": "the response"}}\n\n
  ### Description:{describe}\n\n
  ### Response:"""
  generated_text = llm(prompt)
  ins, res = extract_instruction_response_pairs(generated_text)
  All_instructions.extend(ins)
  All_reponses.extend(res)

100%|██████████| 60/60 [02:30<00:00,  2.51s/it]


In [ ]:
## Generating based on the last plot section
for idx in tqdm(range(40)):
  focus = "last 2 paragraph of plot"
  describe = plot_description
  prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs.
  Make sure the Instruction-Response are in the json format:\n\n
  ### Example: {{"Instruction": "the instruction", "Response": "the response"}}\n\n
  ### Description:{describe}\n\n
  ### Response:"""
  generated_text = llm(prompt)
  ins, res = extract_instruction_response_pairs(generated_text)
  All_instructions.extend(ins)
  All_reponses.extend(res)

100%|██████████| 40/40 [01:38<00:00,  2.46s/it]


In [ ]:
## Generating based on the cast section
for idx in tqdm(range(20)):
  focus = "cast"
  describe = cast_description
  prompt = f"""### Instruction: Based on the {focus} information of the movie "Barbie" below, generate 5 instruction-detailed response pairs.
  Make sure the Instruction-Response are in the json format:\n\n
  ### Example: {{"Instruction": "the instruction", "Response": "the response"}}\n\n
  ### Description:{describe}\n\n
  ### Response:"""
  generated_text = llm(prompt)
  ins, res = extract_instruction_response_pairs(generated_text)
  All_instructions.extend(ins)
  All_reponses.extend(res)

100%|██████████| 20/20 [00:45<00:00,  2.26s/it]


In [ ]:
len(All_reponses)

954

In [ ]:
df = pd.DataFrame()
df["Intructions"] = All_instructions
df["Responses"] = All_reponses

In [ ]:
df.to_csv("Barbie_ChatGPT_genAI.csv", index=False)

In [ ]:
df.head()

,Intructions,Responses
0,Who directed the movie Barbie?,The movie Barbie was directed by Greta Gerwig.
1,Who wrote the screenplay for the movie Barbie?,Greta Gerwig and Noah Baumbach wrote the scree...
2,Who stars as the title character in the movie ...,Margot Robbie stars as the title character in ...
3,Where was the movie Barbie primarily filmed?,The movie Barbie was primarily filmed at Warne...
4,When was the movie Barbie released in the Unit...,The movie Barbie was released in the United St...


In [ ]:
df.shape

(954, 2)

So, yeah, life is indeed not perfect. We got 954 data, although we expected 1000. Still, it's a 95.4% success, right ? ;)

Also, fyi, I spent $.27 for this